# Quality control - BSRN checks

The most commonly applied quality control (QC) checks are the [BSRN recommended QC tests V2](https://bsrn.awi.de/fileadmin/user_upload/bsrn.awi.de/Publications/BSRN_recommended_QC_tests_V2.pdf) (these are the QC checks introduced in the lecture notes). The advantage of such automatic checks is that they are easy to implement, applicable to stations in most climates, and do not require extensive experience of the analyst.

In this exercise, you will learn how to apply the BSRN quality to solar irradiance measurements.

In [ ]:
# Install pvlib on Google Colab as this is not a standard package.
!pip install pvlib

In [2]:
import pvlib  # library for PV and solar calculations
import pandas as pd  # library for data analysis
import matplotlib.pyplot as plt  # library for plotting
import numpy as np  # library for math and linear algebra

## Step 1: Read dataset
The example dataset contains data of GHI, DHI, and DNI from DTU's Solar Radiation Monitoring Station for August 2025.

Note, a number of severe but plausible errors have been added to the dataset!

In [ ]:
file_url = 'https://raw.githubusercontent.com/AdamRJensen/solar-resource-course/refs/heads/main/data/example_dataset_with_errors.csv'

data = pd.read_csv(file_url, index_col=[0], parse_dates=[0])

data.head()

## Step 2: Calculate extraterrestrial irradiance

Use the pvlib-python function [``pvlib.irradiance.get_extra_radiation``](https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.irradiance.get_extra_radiation.html) to calculate extraterrestrial irradiance.

*Hint: for the ``datetime_or_doy`` input to the function you can simply use ``data.index``.*

In [ ]:
# write your code here
data['ETR'] = 

## Step 3: Calculate the BSRN limits

In this exercise, we will be using the **extremely rare limits**. The equations for the limits can be found in the lecture note for Lesson 3.

The first step is to calculate the upper limits. We don't need to calculate the lower limits, as they are simply a fixed value.

In [ ]:
# Write the code for the upper limits:

data['ghi_upper_limit'] = 
data['dhi_upper_limit'] = 
data['dni_upper_limit'] = 

## Step 4: Calculate limit flags
Once the limits have been derived, we can now compute the limit flags for each component. Remember that a flag should be True (1) if the measurement is suspicious or False (0) if the measurement seems plausible.

*Hint: Since we are comparing two conditions (whether the irradiance is above or below a threshold), we need to use either the ``&`` for AND if both conditions need to be true or   ``|`` for OR if only one condition needs to be true.*

In [ ]:
data['ghi_limit_flag'] = (data['ghi'] < -4) | (data['ghi'] > data['ghi_upper_limit'])
data['dhi_limit_flag'] = 
data['dni_limit_flag'] = 

## Step 5: Determine calculated GHI

For the comparison QC checks we need the calculated GHI from DHI and DNI (remember the closure equation?).

In [ ]:
# Write your code here
data['ghi_calc'] = 

## Step 6: Closure check

The most powerful QC check available is the closure check because it allows us to compare the same value (GHI) obtained from two independent sources.

In the below code cell, you should calculated the QC flag for the closure check.

*Hint: To make it easier to implement, you can calculate two flags, one for high zenith angles and one for low zenith angles.*

In [ ]:
# Calculate the closure check flag
data['closure_high_zenith_flag'] = 
data['closure_low_zenith_flag'] = 

## Step 7: Diffuse fraction check

The second BSRN comparison check assesses the diffuse fraction:

$K = DHI / GHI$

Similar to the closure check above, determine the diffuse fraction QC flag for the high and low zenith regimes.

In [6]:
# Write your code here


## Step 8: Calculated statistics

In the last step, you need to calculate the percentage of flagged data for each QC flag:

In [7]:
# Write your code here


## EXTRA

You have now successfully applied the BSRN QC checks and significantly enhanced your dataset by removing potentially erroneous data.

However, to improve future data collection, it is necessary to understand what went wrong with our measurements.

Therefore, the final step is to visualize the data and figure out what went wrong. Note, 5 different errors were added to the dataset!

In [4]:
# Explore the data visually here